In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

In [4]:
from data_processing_helper import *

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [30]:
filepath = '/Users/yunyihuang/Desktop/georgelab/gl_local/data/BSB273EC13HSOXYLGA14_output.xlsx'
df = cleanup(filepath)
df_filtered = filtered_reward(df)
reward_histogram(df_filtered)

/Users/yunyihuang/Desktop/georgelab/gl_local/data/BSB273EC13HSOXYLGA14_output.xlsx
CLEANING COMPLETED


In [32]:
print('Inter-Reward-Intervals Dataframe')
df_filtered[['Subject', 'Latency', 'cleanedIntervals','meanInterval', 'stdInterval']]

Inter-Reward-Intervals Dataframe


,Subject,Latency,cleanedIntervals,meanInterval,stdInterval
0,M1369,18,"[98.0, 190.0, 67.0, 76.0, 432.0, 658.0, 45.0, ...",329.54,432.24
1,M1370,425,"[1005.0, 154.0, 2608.0, 26.0, 1994.0, 2174.0, ...",1332.15,1484.10
2,M1371,3596,"[1600.0, 1056.0, 227.0, 55.0, 1080.0, 312.0, 2...",672.42,485.51
3,M1372,450,"[372.0, 1443.0, 1430.0, 1804.0, 1468.0, 1812.0...",1197.30,873.70
4,M1373,1,"[297.0, 603.0, 815.0, 105.0, 895.0, 311.0, 372...",724.57,1123.04
5,M1374,369,"[38.0, 470.0, 471.0, 662.0, 54.0, 21.0, 23.0, ...",1076.11,1671.98
6,M1375,12,"[179.0, 680.0, 780.0, 570.0, 791.0, 562.0, 623...",795.84,642.96
7,M1376,43,"[545.0, 660.0, 145.0, 54.0, 6451.0, 809.0, 699...",3578.10,4067.57
8,M1377,1351,"[820.0, 23.0, 540.0, 29.0, 654.0, 34.0, 31.0, ...",431.46,640.25
9,M1378,47,"[71.0, 189.0, 86.0, 791.0, 1017.0, 413.0, 56.0...",792.55,1760.50


In [24]:
intervals_cohort = list(itertools.chain.from_iterable(df_filtered.cleanedIntervals.tolist()))
print('cohort mean: ' + str(np.mean(intervals_cohort)))
print('cohort std: ' + str(np.std(intervals_cohort)))

cohort mean: 663.0932944606413
cohort std: 1126.19883906679


In [28]:
# return a df that contains reward latency, all rewards, intervals, and cleaned intervals
def filtered_reward(df):
    filtered_cols = ['Subject'] + [col for col in df.columns if 'Reward ' in col]
    df_reward = df[filtered_cols].sort_values('Subject').reset_index().drop('index',axis=1)
    df_reward['allRewards'] = df_reward.iloc[:,1:].values.tolist()
    df_reward['Latency'] = df_reward['Reward 1']
    df_filtered = df_reward[['Subject', 'Latency', 'allRewards']]
    df_filtered['Intervals'] = df_filtered['allRewards'].apply(lambda lst:[j-i for i, j in zip(lst[:-1], lst[1:])])
    df_filtered['cleanedIntervals'] = df_filtered['Intervals'].apply(lambda lst: [lst[0]]+[val for val in lst[1:] if val > 0][:-1])
    df_filtered['meanInterval'] = df_filtered['cleanedIntervals'].apply(lambda x: round(np.mean(x), 2))
    df_filtered['stdInterval'] = df_filtered['cleanedIntervals'].apply(lambda x: round(np.std(x), 2))
    
    return df_filtered

In [29]:
# draw histogram of the filtered reward dataframe
def reward_histogram(df_filtered):
    data = []
    for s in df_filtered['Subject']:
        cl = df_filtered[df_filtered['Subject']==s]['cleanedIntervals'].values[0]
        for val in cl:
            data.append([s,val])
    df_dist=pd.DataFrame(data,columns=['ID','Second'])
    numBins = round(max(df_dist['Second'])/20)
    
    fig = px.histogram(df_dist, 
                   x="Second", 
                   color="ID", 
                   nbins = numBins,
                   #marginal="box", # can be `box`, `violin`
                   hover_data=df_dist.columns,
                   color_discrete_sequence=px.colors.qualitative.Dark24,
                   title='Inter-Reward Interval Distribution')

    fig.update_layout(autosize=False,
                    width=900,
                    height=700,
                    margin=dict(l=50,
                                r=50,
                                b=100,
                                t=100,
                                pad=4))
    fig.show()
    
    return None

### scratch

In [2]:
filtered_cols = ['Subject'] + [col for col in df.columns if 'Reward ' in col]
df_reward = df[filtered_cols].sort_values('Subject').reset_index().drop('index',axis=1)
df_reward

NameError: name 'df' is not defined

In [372]:
df_reward['allRewards'] = df_reward.iloc[:,1:].values.tolist()
df_reward.head()

,Subject,Reward 1,Reward 2,Reward 3,Reward 4,Reward 5,Reward 6,Reward 7,Reward 8,Reward 9,...,Reward 194,Reward 195,Reward 196,Reward 197,Reward 198,Reward 199,Reward 200,Reward 201,allRewards,Latency
0,F1317,19,47,84,688,1046,1123,1504,2085,2117,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[19.0, 47.0, 84.0, 688.0, 1046.0, 1123.0, 1504...",19
1,F1318,183,635,1210,1536,1781,1809,2447,2477,2954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[183.0, 635.0, 1210.0, 1536.0, 1781.0, 1809.0,...",183
2,F1319,57,95,235,608,713,1370,1459,2132,3009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[57.0, 95.0, 235.0, 608.0, 713.0, 1370.0, 1459...",57
3,F1321,13,622,657,1368,1504,2626,2776,3263,3287,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[13.0, 622.0, 657.0, 1368.0, 1504.0, 2626.0, 2...",13
4,F1322,1489,3166,3240,3463,3494,5054,5385,5670,7896,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1489.0, 3166.0, 3240.0, 3463.0, 3494.0, 5054....",1489


In [373]:
df_filtered = df_reward[['Subject', 'Latency', 'allRewards']]
df_filtered['Intervals'] = df_filtered['allRewards'].apply(lambda lst:[j-i for i, j in zip(lst[:-1], lst[1:])])
df_filtered['cleanedIntervals'] = df_filtered['Intervals'].apply(lambda lst: [val for val in lst if val > 0][:-1])
df_filtered

,Subject,Latency,allRewards,Intervals,cleanedIntervals
0,F1317,19,"[19.0, 47.0, 84.0, 688.0, 1046.0, 1123.0, 1504...","[28.0, 37.0, 604.0, 358.0, 77.0, 381.0, 581.0,...","[28.0, 37.0, 604.0, 358.0, 77.0, 381.0, 581.0,..."
1,F1318,183,"[183.0, 635.0, 1210.0, 1536.0, 1781.0, 1809.0,...","[452.0, 575.0, 326.0, 245.0, 28.0, 638.0, 30.0...","[452.0, 575.0, 326.0, 245.0, 28.0, 638.0, 30.0..."
2,F1319,57,"[57.0, 95.0, 235.0, 608.0, 713.0, 1370.0, 1459...","[38.0, 140.0, 373.0, 105.0, 657.0, 89.0, 673.0...","[38.0, 140.0, 373.0, 105.0, 657.0, 89.0, 673.0..."
3,F1321,13,"[13.0, 622.0, 657.0, 1368.0, 1504.0, 2626.0, 2...","[609.0, 35.0, 711.0, 136.0, 1122.0, 150.0, 487...","[609.0, 35.0, 711.0, 136.0, 1122.0, 150.0, 487..."
4,F1322,1489,"[1489.0, 3166.0, 3240.0, 3463.0, 3494.0, 5054....","[1677.0, 74.0, 223.0, 31.0, 1560.0, 331.0, 285...","[1677.0, 74.0, 223.0, 31.0, 1560.0, 331.0, 285..."
5,F1323,454,"[454.0, 3328.0, 3382.0, 4523.0, 4545.0, 4600.0...","[2874.0, 54.0, 1141.0, 22.0, 55.0, 2079.0, 50....","[2874.0, 54.0, 1141.0, 22.0, 55.0, 2079.0, 50...."
6,F1324,19,"[19.0, 103.0, 208.0, 1594.0, 1777.0, 1977.0, 2...","[84.0, 105.0, 1386.0, 183.0, 200.0, 58.0, 340....","[84.0, 105.0, 1386.0, 183.0, 200.0, 58.0, 340...."
7,F1325,714,"[714.0, 747.0, 1181.0, 1309.0, 1817.0, 1840.0,...","[33.0, 434.0, 128.0, 508.0, 23.0, 43.0, 22.0, ...","[33.0, 434.0, 128.0, 508.0, 23.0, 43.0, 22.0, ..."
8,F1326,989,"[989.0, 1030.0, 1399.0, 1422.0, 1496.0, 1519.0...","[41.0, 369.0, 23.0, 74.0, 23.0, 22.0, 893.0, 6...","[41.0, 369.0, 23.0, 74.0, 23.0, 22.0, 893.0, 6..."
9,F1327,46,"[46.0, 81.0, 459.0, 1478.0, 1761.0, 2218.0, 26...","[35.0, 378.0, 1019.0, 283.0, 457.0, 449.0, 571...","[35.0, 378.0, 1019.0, 283.0, 457.0, 449.0, 571..."


In [376]:
data = []
for s in df_filtered['Subject']:
    cl = df_filtered[df_filtered['Subject']==s]['cleanedIntervals'].values[0]
    for val in cl:
        data.append([s,val])

In [377]:
df_dist=pd.DataFrame(data,columns=['ID','Second'])
df_dist

,ID,Second
0,F1317,28.0
1,F1317,37.0
2,F1317,604.0
3,F1317,358.0
4,F1317,77.0
...,...,...
1512,M1352,23.0
1513,M1352,130.0
1514,M1352,1610.0
1515,M1352,43.0


In [380]:
numBins = round(max(df_dist['Second'])/20)
numBins

288

In [388]:
fig = px.histogram(df_dist, 
                   x="Second", 
                   color="ID", 
                   nbins = numBins,
                   marginal="box", # can be `box`, `violin`
                   hover_data=df_dist.columns,
                   color_discrete_sequence=px.colors.qualitative.Dark24,
                   title='Inter-Reward Interval Distribution')

fig.update_layout(autosize=False,
                width=900,
                height=700,
                margin=dict(l=50,
                            r=50,
                            b=100,
                            t=100,
                            pad=4))


fig.show()